간단한 conv net 만들기
  - 데이터는 mnist 사용
  - conv - pool - conv - pool -conv - faltten - fc

In [1]:
import tensorflow as tf
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [2]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

train_labels = tf.keras.utils.to_categorical(train_labels,10)
test_labels = tf.keras.utils.to_categorical(test_labels,10)


11490434/11490434 [==============================] - 0s 0us/step


In [3]:
model.compile(loss = tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])
hist = model.fit(train_images,train_labels,epochs=10,batch_size=128,validation_data=(test_images,test_labels))

Epoch 1/10
469/469 [==============================] - 16s 8ms/step - loss: 0.2135 - accuracy: 0.9329 - val_loss: 0.0691 - val_accuracy: 0.9764
Epoch 2/10
469/469 [==============================] - 3s 6ms/step - loss: 0.0544 - accuracy: 0.9832 - val_loss: 0.0416 - val_accuracy: 0.9860
Epoch 3/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0366 - accuracy: 0.9887 - val_loss: 0.0299 - val_accuracy: 0.9897
Epoch 4/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0277 - accuracy: 0.9913 - val_loss: 0.0264 - val_accuracy: 0.9923
Epoch 5/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0213 - accuracy: 0.9936 - val_loss: 0.0391 - val_accuracy: 0.9880
Epoch 6/10
469/469 [==============================] - 3s 6ms/step - loss: 0.0175 - accuracy: 0.9947 - val_loss: 0.0284 - val_accuracy: 0.9918
Epoch 7/10
469/469 [==============================] - 3s 7ms/step - loss: 0.0134 - accuracy: 0.9960 - val_loss: 0.0267 - val_accuracy: 0.9923
Epoch

dog vs cat 분류

In [ ]:
!unzip /content/drive/MyDrive/imgs/dogs-vs-cats.zip -d ./dogcat

In [ ]:
!unzip /content/dogcat/test1.zip -d /content/dogcat/test
!unzip /content/dogcat/train.zip -d /content/dogcat/train

In [6]:
import tensorflow as tf
inputs = tf.keras.Input(shape=(180, 180, 3))
x = tf.keras.layers.Rescaling(1./255)(inputs)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [7]:
model.compile(loss = tf.keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [24]:
!mkdir /content/dogcat/train/dog
!mkdir /content/dogcat/train/cat

In [26]:
from glob import glob
import shutil
cat_path = '/content/dogcat/train/cat/'
dog_path = '/content/dogcat/train/dog/'
filepahts = glob("/content/dogcat/train/train/*.*")
for idx,filename in enumerate( [name.split("/")[-1] for name in filepahts]):
  if 'dog' in filename:
    shutil.copyfile(src =filepahts[idx] , dst =dog_path+filename )
  elif 'cat' in filename:
    shutil.copyfile(src =filepahts[idx] , dst =cat_path+filename )

shutil.rmtree('/content/dogcat/train/train')

In [33]:
!mkdir /content/dogcat/test/dog
!mkdir /content/dogcat/test/cat

In [34]:
from glob import glob
import shutil
cat_path = '/content/dogcat/test/cat/'
dog_path = '/content/dogcat/test/dog/'
filepahts = glob("/content/dogcat/test/test1/*.*")
for idx,filename in enumerate( [name.split("/")[-1] for name in filepahts]):
  if 'dog' in filename:
    shutil.copyfile(src =filepahts[idx] , dst =dog_path+filename )
  elif 'cat' in filename:
    shutil.copyfile(src =filepahts[idx] , dst =cat_path+filename )
shutil.rmtree('/content/dogcat/test/test1')

In [35]:
from tensorflow.keras.utils import image_dataset_from_directory

In [36]:
train_data_set = image_dataset_from_directory(
    "/content/dogcat/train",
    image_size = (180,180),
    batch_size=32
)
test_data_set = image_dataset_from_directory(
    "/content/dogcat/test",
    image_size = (180,180),
    batch_size=32
)

Found 25000 files belonging to 2 classes.
Found 0 files belonging to 2 classes.


ValueError: ignored

In [12]:
for data_batch, label_batch in train_data_set:
  print(f"데이터 배치 크기 : {data_batch.shape}")
  print(f"라벨 배치 크기 : {label_batch.shape}")
  break

데이터 배치 크기 : (32, 180, 180, 3)
라벨 배치 크기 : (32,)


In [ ]:
hist = model.fit(
   train_data_set,
   epochs=30,
   validation_data= test_data_set
)